<p style="font-size: 12px">En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.</p>

<div hidden>
    <!-- Font normal -->
    <p style="font-size: 16px">text</p>
    <!-- Font pequeño -->
    <p style="font-size: 15px">text</p>
    <!-- hr interno -->
    <hr style="height: 1px; color: hsl(221.25 24.24% 18%); background: hsl(221.25 24.24% 18%); font-size: 0; border: 0">
    <!-- hr externo -->
    <hr style="height: 1px; color: hsl(163.3 98.98% 58.43%); background: hsl(163.3 98.98% 58.43%); font-size: 0; border: 0">
    <b style="color: hsl(163.3 98.98% 58.43%)">bold</b>
</div>

<h1 align="center">LATAM Data Engineer Challenge</h1>
<hr style="height: 1px; color: hsl(221.25 24.24% 18%); background: hsl(221.25 24.24% 18%); font-size: 0; border: 0">
<p style="font-size: 16px">Hola! gracias por leer! A continuación paso a detallar los pasos realizados y todo lo pensado para este desafio.</p>

<div style="background-color: hsl(221.25 24.24% 18%); width: 30%; padding:5px 10px; border-radius: 5px">
<h3>Tech usadas</h3>
<ul>
<li>PySpark v3.5.1</li>
<li>Python v3.9.13</li>
<li>Pandas v2.2.2</li>
</ul>
</div>


<h2>Proceso mental</h2>
<p style="font-size: 16px">Después de revisar la estructura del dataset y revisar la documentación, pensé que lo primero que debía hacer era buscar una manera de unir todos los <b style="color: hsl(163.3 98.98% 58.43%)">quotedTweets</b> ya que finalmente correpsonden a otro tweet, y con esto lograr un mayor dataset.</p>

<p style="font-size: 16px">Siguiendo, y como quería usar PySpark creé una clase que instanciara a <b style="color: hsl(163.3 98.98% 58.43%)">Spark</b> para dejar ahí toda la configuración y cualquier método que me fuera útil. Cabe descatar que todo el proceso lo corrí en local, por ende fui probando distintas configuraciones de <b style="color: hsl(163.3 98.98% 58.43%)">Spark</b>, según ciertos errores que me iban arrojando las ejecuciones de código.</p>

<p style="font-size: 16px">Principalmente sufrí con un par de errores del tipo: "<i>Py4JJavaError: An error occurred while calling {ABC}</i>". Esto ocurría casi todas las veces que usé UDFs, busqué varias soluciones pero no dí con ninguna así que opté por no usar UDFs. Pienso que probablemente tengo algo mal seteado en Spark, y por ello me arrojó error cada vez que quise usd UDFs.</p>

<p style="font-size: 16px">Luego de realizar algunas pruebas con el archivo JSON, preferí pasarlo a Parquet para así mejorar el rendimiento de las ejecuciones.</p>

<br>
<br>
<hr style="height: 1px; color: hsl(221.25 24.24% 18%); background: hsl(221.25 24.24% 18%); font-size: 0; border: 0">

<p style="font-size: 14px">Referencias</p>
<ul>
<li><a href="https://medium.com/@dipan.saha/getting-started-with-pyspark-day-1-37e5e6fdc14b" style="color: hsl(163.3 98.98% 58.43%)" target="_blank">Instalar PySpark en windows</a></li>
<li><a href="https://spark.apache.org/docs/3.0.2/configuration.html" style="color: hsl(163.3 98.98% 58.43%)" target="_blank">Docs de configuración de PySpark</a></li>
</ul>

<hr style="height: 1px; color: hsl(163.3 98.98% 58.43%); background: hsl(163.3 98.98% 58.43%); font-size: 0; border: 0">

<h2>Inicialización de variables</h2>

<p style="font-size: 16px">Creé la celda siguiente para chequear la instalación de Java.</p>

In [ ]:
# check de java
from subprocess import check_output, STDOUT
try:
  check_output("java -version", stderr=STDOUT, shell=True).decode('utf-8')
except OSError:
  # raise KeyboardInterrupt("Por favor instale Java JDK para continuar.")
  print("Java no está instalado en su PC, para que el programa funcione correctamente, por favor instale Java en su PC.")

<p style="font-size: 16px">Luego inicializo los nombres de archivos y las rutas, además de preguntar si es que existe el .zip para descomprimirlo en la carpeta <b style="color: hsl(163.3 98.98% 58.43%)">data</b></p>

In [ ]:
import os
import cProfile
import pstats
from zipfile import ZipFile

# declaro nombres de archivo y rutas
json_file = "farmers-protest-tweets-2021-2-4.json"
parquet_file = "tweets.parquet"
data_folder_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
json_path = f"{data_folder_path}\\{json_file}"
parquet_path = f"{data_folder_path}\\{parquet_file}"

# checkea si existe la carpeta data, sino, la crea
if not os.path.exists(data_folder_path):
    os.makedirs(data_folder_path)

# checkea si existe el archivo tweets.json.zip, si no, checkea si existe el archivo json, si no, lanza excepción
if not os.path.exists(f"{data_folder_path}\\tweets.json.zip"):
    if not os.path.exists(f"{data_folder_path}\\{json_file}"):
        raise FileNotFoundError("Por favor descargue el archivo 'tweets.json.zip' y colóquelo en la carpeta 'data'")

# si no existe el archivo json, descomprime el archivo zip
if not os.path.exists(f"{data_folder_path}\\{json_file}"):
    with ZipFile(f"{data_folder_path}\\tweets.json.zip", 'r') as zObject: 
        zObject.extractall(path=f"{data_folder_path}")

<p style="font-size: 16px">Luego pregunta si es que existe el archivo parquet, sino, lo crea.</p>

<p style="font-size: 16px">La función <b style="color: hsl(163.3 98.98% 58.43%)">json_to_parquet</b> toma el json principal (el de 389mb) y lo "desnestea". Esto va a buscar si existe algún elemento <b style="color: hsl(163.3 98.98% 58.43%)">quotedTweet</b>, y si existe une sus datos con el dataframe principal, esto para crear un gran dataset con todos los tweets, luego de ellos lo pasa a formato <b style="color: hsl(163.3 98.98% 58.43%)">parquet</b>.</p>

```python
def json_to_parquet(ruta_del_json: str, ruta_del_parquet: str) -> None
```

In [ ]:
from helpers.helperFuncs import json_to_parquet
# si no encuentra el archivo parquet, lo crea
if not os.path.exists(f"{parquet_path}"):
    json_to_parquet(json_path, parquet_path)

<hr style="height: 1px; color: hsl(163.3 98.98% 58.43%); background: hsl(163.3 98.98% 58.43%); font-size: 0; border: 0">

#### Pregunta 1
Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. Debe incluir las siguientes funciones:
```python
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
Returns: 
[(datetime.date(1999, 11, 15), "LATAM321"), (datetime.date(1999, 7, 15), "LATAM_CHI"), ...]
```

In [ ]:
#%%script false --no-raise-error
from q1_memory import q1_memory
q1_mem_result = q1_memory(f"{parquet_path}")
q1_mem_result

In [ ]:
#%%script false --no-raise-error
from q1_time import q1_time

with cProfile.Profile() as pr:
    q1_time_result = q1_time(f"{parquet_path}")

q1_stats = pstats.Stats(pr).strip_dirs().sort_stats('time')
q1_stats.dump_stats('./stats/q1_time')
q1_stats.print_stats()
q1_time_result

<hr style="height: 1px; color: hsl(163.3 98.98% 58.43%); background: hsl(163.3 98.98% 58.43%); font-size: 0; border: 0">

#### Pregunta 2
Los top 10 emojis más usados con su respectivo conteo. Debe incluir las siguientes funciones:
```python
def q2_time(file_path: str) -> List[Tuple[str, int]]:
```
```python
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
```
```python
Returns: 
[("✈️", 6856), ("❤️", 5876), ...]
```

In [ ]:
#%%script false --no-raise-error
from q2_memory import q2_memory
q2_mem_result = q2_memory(f"{parquet_path}")
q2_mem_result

In [ ]:
#%%script false --no-raise-error
from q2_time import q2_time

with cProfile.Profile() as pr:
    q2_time_result = q2_time(f"{parquet_path}")

q2_stats = pstats.Stats(pr).strip_dirs().sort_stats('time')
q2_stats.dump_stats('./stats/q2_time')
q2_stats.print_stats()
q2_time_result

<hr style="height: 1px; color: hsl(163.3 98.98% 58.43%); background: hsl(163.3 98.98% 58.43%); font-size: 0; border: 0">

#### Pregunta 3
El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos. Debe incluir las siguientes funciones:
```python
def q3_time(file_path: str) -> List[Tuple[str, int]]:
```
```python
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
```
```python
Returns: 
[("LATAM321", 387), ("LATAM_CHI", 129), ...]
```

In [ ]:
#%%script false --no-raise-error
from q3_memory import q3_memory
q3_mem_result = q3_memory(f"{parquet_path}")
q3_mem_result

In [ ]:
#%%script false --no-raise-error
from q3_time import q3_time

with cProfile.Profile() as pr:
    q3_time_result = q3_time(f"{parquet_path}")

q3_stats = pstats.Stats(pr).strip_dirs().sort_stats('time')
q3_stats.dump_stats('./stats/q3_time')
q3_stats.print_stats()
q3_time_result